In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from itertools import product
import os, sys

In [ ]:
sectors=['CAMEL']
recipes=['adding_up', 'risk_aversion',]
discs=['euler_ramsey', 'constant']
kinds=['p1', 'p99', 'zeroes']
eta_rhos = {
    2.0 : 0.0,
    1.016010255: 9.149608e-05,
    1.244459066: 0.00197263997,
    1.421158116: 0.00461878399,
    1.567899395: 0.00770271076,
}

In [ ]:
def get_sim_id(sector,
               recipe,
               disc,
               eta,
               rho,
               kind,
              ):

    results = f"/mnt/CIL_integration/menu_results_AR6_epa/{sector}/2020/unmasked"
    
    sccs = xr.open_dataset(
            f"{results}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_sccs.nc4"
        ).sel(weitzman_parameter='0.5', fair_aggregation='uncollapsed', drop=True).uncollapsed_sccs
    
    if disc=='constant':
        sccs = sccs.sel(discrate=0.02)

    if kind=='p1':
        cutoff = sccs.quantile(0.01, ['simulation'])
        rff_ids = sccs.where(sccs < cutoff, drop=True)
    elif kind=='zeroes':
        rff_ids = sccs.where((sccs >-0.01) & (sccs<0.01), drop=True)
    elif kind=='p99':
        cutoff = sccs.quantile(0.99, ['simulation'])
        rff_ids = sccs.where(sccs > cutoff, drop=True)

    rff_ids = rff_ids.to_dataframe()

    csv = rff_ids.loc[~rff_ids.uncollapsed_sccs.isnull()]
 
    os.makedirs(f"{results}/sim_ids/", exist_ok=True)
    csv.to_csv(f"{results}/sim_ids/{recipe}_{disc}_eta{eta}_rho{rho}_{kind}_sim_ids.csv")

In [ ]:
for sector, recipe, disc, er, kind in product(sectors, recipes, discs, eta_rhos.items(), kinds):
    print(sector, recipe, disc, er, kind)
    get_sim_id(sector,
               recipe,
               disc,
               er[0],
               er[1],
               kind,
              )